<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/master/Class_06_2_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# BIO 1173: Intro Computational Biology

**Module 6: Convolutional Neural Networks (CNN) for Computer Vision**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Integrative Biology](https://sciences.utsa.edu/integrative-biology/), [UTSA](https://www.utsa.edu/)

### Module 6 Material

* Part 6.1: Using Convolutional Neural Networks 
* **Part 6.2: Using Pretrained Neural Networks with Keras** 
* Part 6.3: Looking at Keras Generators and Image Augmentation


## Using Google COLAB

In order to run this lesson on Google COLAB, you will already need to have a [Google Drive account](https://support.google.com/a/users/answer/13022292?hl=en).

When you open this lesson in COLAB, you must immediately change the `RUNTIME` environment to take advantage of GPU **_before_** you start your lesson. While you can change your `RUNTIME` environment after you start, you will to restart from the beginning. 

When you open up this lesson in COLAB, you should select `RUNTIME -> Change runtime type` 

![___](https://biologicslab.co/BIO1173/images/class_06/class_06_2_COLAB1.png)



This will open the following popup menu:

![___](https://biologicslab.co/BIO1173/images/class_06/class_06_2_COLAB2.png)


Select `T4 GPU` and then `Save`. 

Make sure to save a copy of your lesson to your Google Drive. And don't forget, if you stop working on your lesson, COLAB will "kick you off" after some period of inactivity.

So if you decide you want to run this lesson on Google COLAB, click on this button:

### Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.
  Running the following code will map your GDrive to ```/content/drive```.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

### Lesson Setup

Run the next code cell to load necessary packages

In [ ]:
# You MUST run this code cell first
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import pandas as pd

import os
import shutil
path = '/'
memory = shutil.disk_usage(path)
LESSON_DIRECTORY = os.getcwd()
print("Your LESSON_DIRECTORY is: " + LESSON_DIRECTORY)

### Define functions

The cell below creates the function(s) needed for this lesson.

In [ ]:
# Simple function to print out elasped time
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# Part 6.2: Transfer Learning for Computer Vision

Many advanced prebuilt neural networks are available for computer vision, and Keras provides direct access to many networks. **_Transfer Learning_** is the technique where you use these prebuilt neural networks. 

There are several different levels of transfer learning.

* Use a prebuilt neural network in its entirety
* Use a prebuilt neural network's structure
* Use a prebuilt neural network's weights

We will begin by using the **_MobileNet_** prebuilt neural network in its entirety. MobileNet will be loaded and allowed to classify simple images. We can already classify 1,000 images through this technique without ever having trained the network.

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow.keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

We begin by downloading weights for a MobileNet trained for the imagenet dataset, which will take some time to download the first time you train the network.

In [ ]:
# Download Pre-Trained neural network

model = MobileNet(weights='imagenet',include_top=True)

The loaded network is a Keras neural network. However, this is a neural network that a third party engineered on advanced hardware. Merely looking at the structure of an advanced state-of-the-art neural network can be educational.

In [ ]:
model.summary()

Several clues to neural network architecture become evident when examining the above structure.

**_MobileNet_** is a neural network architecture designed for mobile and embedded applications to be computationally efficient. The key components of the MobileNet architecture include depthwise separable convolutions and pointwise convolutions.
* **Depthwise separable convolutions:** MobileNet replaces traditional convolutions with depthwise separable convolutions, which consists of two separate operations: depthwise convolutions and pointwise convolutions. Depthwise convolutions apply a single filter for each input channel, while pointwise convolutions apply 1x1 convolutions to combine the outputs of depthwise convolutions.
* **Pointwise convolutions:** Pointwise convolutions are used to increase the depth of the feature maps while keeping the spatial dimensions the same. This allows for efficient learning of complex patterns while reducing the computational cost.
* **Inverted residuals with linear bottleneck:** MobileNetV2 introduces inverted residuals with linear bottleneck to improve performance. Inverted residuals use an expansion layer to increase the number of channels followed by a depthwise convolution and a projection layer to reduce the number of channels back to the original dimensions.

Overall, the MobileNet architecture is designed to be lightweight and efficient while maintaining high accuracy for a wide range of tasks, making it ideal for deployment on mobile and edge devices.

## Example 1: Use MobileNet to classify images

The code in the cell below creates a two functions that we will need to use classify images using MobileNet. 

* **make_square()** Since MobileNet is designed to classify images with the same number of horizontal and vertical pixels (i.e. a 'square' image), this function uses a combination of padding and cropping to convert any image into a 'square` image.
  
* **classify_image()** This function does most of the work. It first retrives the image from the HTTPS server and resizes it before processing it by the MobileNet model that we previously downloaded. The actual prediction is made by this line of code:

~~~text
  # Use MobileNet model to predict image
  pred = model.predict(x)
~~~

We will now use the MobileNet to classify several image URLs below.  You are encourged to to add additional URLs of your own to see how well the MobileNet can classify.

In [ ]:
# Example 1: Use MobileNet to classify images

%matplotlib inline
from PIL import Image, ImageFile
from matplotlib.pyplot import imshow
import requests
import numpy as np
from io import BytesIO
from IPython.display import display, HTML
from tensorflow.keras.applications.mobilenet import decode_predictions

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 3

# Define HTTPS image source 
ROOT = "https://biologicslab.co/BIO1173/images/class_06_2/"

# Function to make sample image square
def make_square(img):
    cols,rows = img.size
    
    if rows>cols:
        pad = (rows-cols)/2
        img = img.crop((pad,0,cols,cols))
    else:
        pad = (cols-rows)/2
        img = img.crop((0,pad,rows,rows))
    
    return img
        
# Function to classify image
def classify_image(url):
  x = []
  ImageFile.LOAD_TRUNCATED_IMAGES = False

  # Get image from HTTPS server
  response = requests.get(url)

  # Resize the image to 224 x 224 x 3
  img = Image.open(BytesIO(response.content))
  img.load()
  img = img.resize((IMAGE_WIDTH,IMAGE_HEIGHT),Image.LANCZOS)

  # Additional image processing
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  x = x[:,:,:,:3] # maybe an alpha channel
  
  # Use MobileNet model to predict image
  pred = model.predict(x)

  # Show the image in Jupyterlab/COLAB
  display(img)

  # Print out the image number in MobileNet 
  print(np.argmax(pred,axis=1))

  # Print out MobileNet's first 5 predictions
  lst = decode_predictions(pred, top=5)
  for itm in lst[0]:
      print(itm)


We can now classify an example image.  You can specify the URL of any image you wish to classify.

### Example 1A: Classify images

Our MobileNet model has been trained to recognize a wide range of images. Let's try a few different types of images to get some idea of what MobileNet knows.

Let's start with a picture of Abraham Lincoln. Here is the actual image before being processed:

![___](https://biologicslab.co/BIO1173/images/class_06_2/abraham_lincoln.jpg)

Probably most US school children could correctly identify this image. You should notice that this is _not_ a square image. It is actually 1024 X 576 pixels.

Let's see what happens when we process it and submit the processed image to our model? 

In [ ]:
# Example 1A: 
classify_image(ROOT+"abraham_lincoln.jpg")

Apparently, the creators of MobileNet didn't include images of American presidents in their training set. What MobileNet "saw" wasn't the Lincoln's face, but the clothes he was wearing. MobileNet 'nailed' Lincoln's bow tie with a 95% probability!

We can also see what happens when our code converts a rectangular image (1024 X 576 pixels) into a square image. Lincoln's face is still clearly recognizable albeit a bit 'squashed'. 

### Example 1B: Classify Images

Perhaps MobileNet only recognizes famous people who are currently alive? Let's see how MobileNet does with another former President of the US (POTUS)?


In [ ]:
# Example 1B: 

classify_image(ROOT+"trump.jpg")

Once again, MobileNet doesn't seem to know the names of American Presidents, but seems to focus more on what the person in the image is wearing.  

### Example 1C: Classify Images

Let's try something else. How about a nice Thompson Submachine gun? This particular image is of a [Lancer Tactical Extra 50 Rounds Airsoft Magazine - Airsoft Tommy Thompson Submachine Gun (2X Drum 2X Stick)](https://us.amazon.com/Lancer-Tactical-Rounds-Airsoft-Magazine/dp/B0C24VRLM1) sold on Amazon in case you might want to buy it. 


In [ ]:
# Example 1C:

classify_image(ROOT+"submachine_gun.jpg")

Finally, we have one type of image that MobileNet has been trained on. MobileNet predicted that there was an approximately 54% chance that our image of a Thompson Submachine Gun was an 'assault_rife' and a 49% that the image was some kind of 'rifle'. Technically, Thompson Submachine Guns are _not_ considered to be an assault rife or even a rifle, but it wasn't a bad guess.

### Example 1D: Classify images

Let's see how MobileNet does with an image of an American soldier with a _real_ assault rifle, an M16. 

In [ ]:
# Example 1D: 

classify_image(ROOT+"M16.jpg")

The results are definitely better with a 65% prediction that the image showed an assault rifle, in this case an M16. Again, MobileNet seems to ignore the inclusion of people in an image and focus on inanimate objects in the image. 

## **Exercise 1: Classify images**

For **Exercise 1**, you are present our MobileNet model the following series of 8 animal images. 

1. puffer_fish.jpg (Exercise 1A)
2. king_cobra.jpg (Exercise 1B)
3. monarch_butterfly.jpg (Exercise 1C)
4. viceroy_butterfly.jpg (Exercise 1D)
5. meerkat.jpg (Exercise 1E)
6. paramecium.jpg (Exercise 1F)
7. prairie_dog.jpg (Exercise 1G)
8. great-white-shark.jpg (Exercise 1H)

You will need to make a new code cell for each exercise.

If you are using COLAB, you can add a new code cell by pointing your cursor at the bottom edge of the code cell and pressing the button below: 

![___](https://biologicslab.co/BIO1173/images/class_06_2_add_code.png). 

If your code is correct, you should have received the following output (only the the first value is shown):

1. `puffer_fish.jpg` ('n02655020', 'puffer', 0.99882895)  Note: 99% accurate!
2. `king_cobra.jpg` ('n01748264', 'Indian_cobra', 0.999587) Note: Indian cobras are much smaller than king cobras so technically, MobileNet got this wrong. However, there is no way to know the size of the snake in the image, so we can score this as being 100% accurate.
3. `monarch_butterfly.jpg` ('n02279972', 'monarch', 0.99983335) Note: Nailed it!
4. `viceroy_butterfly.jpg` ('n02279972', 'monarch', 0.9988518) Note: Oops! A total fail, but completely understandable. MobileNet again predicted that there was 100% probability that the picture was a monarch butterfly (_Danaus plexippus_), when in fact it was a viceroy butterfly, a completely different genus and species (_Limenitis archippus_). Of course these two species look very, very similar. The viceroy and the monarch butterflies are [Müllerian mimics](https://en.wikipedia.org/wiki/M%C3%BCllerian_mimicry) of each other. It is certainly possible to train a neural network to do this for example [MonarchNet](https://ai4earthscience.github.io/neurips-2020-workshop/papers/ai4earth_neurips_2020_57.pdf). As will be shown below you could start with the MobileNet as give it additional training to differentiate images of monarchs and viceroys with a high degree of precision.
5. `meerkat.jpg` ('n02138441', 'meerkat', 0.9629027) Note: Not bad! 96% accurate.
6. `paramecium.jpg` ('n01930112', 'nematode', 0.8969467) Note: Not too close. A nematode is round worm such as the common human parasite, the pinworm (_Enterobius_). Apparently, MobileNet wasn't extensively programmed for unicelluar organisms.
7. `prairie_dog.jpg` ('n02361337', 'marmot', 0.93886554) Note: Another fail, but again understandable. [Marmots](https://en.wikipedia.org/wiki/Marmot) genus _Marmota_ are quite similar in appearance to pairie dogs, so we can give a 'pass' to MobileNet for getting these two species confused.
8. `great-white-shark.jpg` ('n01484850', 'great_white_shark', 0.9989716) Note: Nailed it! 100% correct prediction.   

### MobileNet Summary

Overall, our MobileNet neural network did quite well, as long as you picked one of the 1000 image types it supports. For many applications, MobileNet might be entirely acceptable as an image classifier. 

However, if you need to classify very specialized images, like monarch vs viceroy butterflies, or marmots vs prairie dogs --image types supported by imagenet--it is necessary to use **_transfer learning_**.


--------------------------------

## **ResNet vs MobileNet**

MobileNet and ResNet are both popular deep learning architectures, but they have some key differences:

* **Architecture:** MobileNet uses depthwise separable convolutions, which consist of depthwise convolutions and pointwise convolutions, to reduce the computational cost and make the model more efficient for mobile and embedded applications. ResNet, on the other hand, uses residual blocks with skip connections to learn residual mappings for easier training of deep networks.
* **Model size:** MobileNet is known for its lightweight and compact design, making it easy to deploy on mobile devices with limited computational resources. In contrast, ResNet is a deeper network with more parameters, which can lead to higher accuracy but also requires more computational resources.
* **Training complexity:** ResNet can be easier to train compared to MobileNet, especially for deeper architectures, due to the use of skip connections that help with gradient flow and alleviate the vanishing gradient problem.
* **Performance:** ResNet is often used for tasks that require high accuracy, such as image classification on large datasets like ImageNet. MobileNet is designed for efficiency and speed, making it suitable for real-time applications on mobile devices.

Overall, the choice between MobileNet and ResNet depends on the specific requirements of the application, such as computational resources, accuracy goals, and deployment constraints.

----------------------------

## Preparing your computer/lap

In this section, we will train a neural network to count the number of paper clips in images. If you have already completed Class_06_1 using Jupyter Lab (not COLAB) on the computer/laptop that you are using now, you should have already all of the paper clips images stored in a temporary folder, `temp`. 

In class you didn't complete Class_06_1 on the computer/laptop you are using now, or you have deleted these images, the following code will create the necessary folders and download the image data. If these folders have already been created and the image data already downloaded and extracted, you will be informed of this.

### Set ENVIRONMENTAL VARIABLES

The code in the cell below defines a number of ENVIRONMENTAL VARIABLES that are needed for latter code cells. If you are having any problem with this code, uncomment the print statements are re-run this cell.

In [ ]:
# Set ENVIRONMENTAL VARIABLES

URL = "https://biologicslab.co/BIO1173/data/"
DOWNLOAD_SOURCE = URL+"paperclips.zip"
DOWNLOAD_NAME = DOWNLOAD_SOURCE[DOWNLOAD_SOURCE.rfind('/')+1:]
print("DOWNLOAD_SOURCE=",DOWNLOAD_SOURCE)
print("DOWNLOAD_NAME=",DOWNLOAD_NAME)

PATH = "/content"
EXTRACT_TARGET = os.path.join(PATH,"clips")
SOURCE = os.path.join(EXTRACT_TARGET, "paperclips")

print("ENVIRONMENTAL VARIABLES were successfully created.")

If your code is correct, you should see the following output:
~~~text
ENVIROMENTAL VARIABLES were sucessfully created.
~~~

### Download and Extract Image Data

The code in the cell below downloads a zip file from the course HTTPS server, `https://biologicslab.co/BIO1173/data`, called `paperclips.zip` and stores this file to your `/temp` folder. The code then extracts (unzips) data in the zip file into a new folder called `/clips`. 

In [ ]:
# Download and extract the image data
!wget -O {os.path.join(PATH,DOWNLOAD_NAME)} {DOWNLOAD_SOURCE}
!mkdir -p {SOURCE}
!mkdir -p {TARGET}
!mkdir -p {EXTRACT_TARGET}
!unzip -o -j -d {SOURCE} {os.path.join(PATH, DOWNLOAD_NAME)} >/dev/null

### Load the Labels for the Training Set

The labels are contained in a CSV file named **train.csv** for the regression. This file has just two labels, **id** and **clip_count**. The ID specifies the filename; for example, row id 1 corresponds to the file **clips-1.jpg**. The following code loads the labels for the training set and creates a new column, named **filename**, that contains the filename of each image, based on the **id** column.

In [ ]:
# Load the labels for the training set

import pandas as pd

df = pd.read_csv(
        os.path.join(SOURCE,"train.csv"), 
        na_values=['NA', '?'])
    
df['filename']="clips-"+df["id"].astype(str)+".jpg"


This results in the following dataframe.

In [ ]:
df

If your code is correct you should see the following table:

![___](https://biologicslab.co/BIO1173/images/class_06/class_06_2_df.png)



### Split images into training and validation sets

The code in the cell below, splits the paperclips images into training set and a validation set, with 90% of the images going into the training set. The number images in both sets is printed out.

In [ ]:
# Split images into training and validation sets

# create dataframes to data
df_train = pd.read_csv(os.path.join(SOURCE, "train.csv"))
df_train['filename'] = "clips-" + df_train.id.astype(str) + ".jpg"

TRAIN_PCT = 0.9
TRAIN_CUT = int(len(df) * TRAIN_PCT)

df_train = df[0:TRAIN_CUT]
df_validate = df[TRAIN_CUT:]

print(f"Training size: {len(df_train)}")
print(f"Validate size: {len(df_validate)}")

If your code is correct, you should see the following output:
~~~text
Training size: 18000
Validate size: 2000
~~~
We want to use early stopping. To do this, we need a validation set. We will break the data into 80 percent test data and 20 validation. 


## Transfer Learning using `ResNet`
We will make use of the structure of the **_ResNet_** neural network. There are several significant changes that we will make to ResNet to apply to this task. First, ResNet is a classifier; we wish to perform a regression to count. Secondly, we want to change the image resolution that ResNet uses. We will not use the weights from ResNet; changing this resolution invalidates the current weights. Thus, it will be necessary to retrain the network.

Next, we create the generators that will provide the images to the neural network during training. We normalize the images so that the RGB colors between 0-255 become ratios between 0 and 1. We also use the **flow_from_dataframe** generator to connect the Pandas dataframe to the actual image files. We see here a straightforward implementation; you might also wish to use some of the image transformations provided by the data generator.

The **HEIGHT** and **WIDTH** constants specify the dimensions to which the image will be scaled (or expanded). It is probably not a good idea to expand the images.

In [ ]:
# Install Keras package

!pip install keras_preprocessing 

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

WIDTH = 256
HEIGHT = 256

training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  #vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train_cut,
        directory=SOURCE,
        x_col="filename",
        y_col="clip_count",
        target_size=(HEIGHT, WIDTH),
        # Keeping the training batch size small 
        # USUALLY increases performance
        batch_size=32, 
        class_mode='raw')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate_cut,
        directory=SOURCE,
        x_col="filename",
        y_col="clip_count",
        target_size=(HEIGHT, WIDTH),
        # Make the validation batch size as large as you 
        # have memory for
        batch_size=256, 
        class_mode='raw')

If your code is correct, you should see the following output:
~~~text
Found 18000 validated image filenames.
Found 2000 validated image filenames.
~~~
This means that our train and validation generator are working properly and know where to find where in your filesystem your images of paperclips are stored.

### Example 2: Transfer Learning with ResNet

We will now use a ResNet neural network as a basis for our neural network. We will redefine both the input shape and output of the ResNet model, so we will not transfer the weights. Since we redefine the input, the weights are of minimal value. We begin by loading, from Keras, the ResNet50 network. We specify **include_top** as False because we will change the input resolution. We also specify **weights** as false because we must retrain the network after changing the top input layers.

### Example 2: Step 1 - Redefine the input shape

The first modification of the base ResNet model `ResNet50` that we need to make is to redefine the image shape to 256 X 256 pixels. 

In [ ]:
# Example 2: Step 1 - Redefine the input shape

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input

# Set variables
HEIGHT = 256
WIDTH = 256

input_tensor = Input(shape=(HEIGHT, WIDTH, 3))

base_model = ResNet50(
    include_top=False, weights=None, input_tensor=input_tensor,
    input_shape=None)

### Example 2: Step 2 - Add layers to convert model to regression

Now we must add a few layers to the end of the neural network so that it becomes a regression model. As you should expect for a regression model, there is only a single neuron in the output layer:
~~~text
model=Model(inputs=base_model.input,outputs=Dense(1)(x))
~~~

In [ ]:
# Example 2: Step 2 - Add layers to convert model to regression

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
model=Model(inputs=base_model.input,outputs=Dense(1)(x))


### Example 2: Step 3 - Train the model

In the cell below, we provide **_additional_** training to the base `ResNet50` model by "showing it" 16200 test images with their labels (i.e. how many paperclips are in the image) using a 1800 validation image set to allow us to have EarlyStopping. 

Training is like before. However, we do not define the entire neural network here.

In [ ]:
# Example 2: Step 3 - Train the model
import time

# Set variables
EPOCHS=1 #00

start_time=time.time()

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError

# Important, calculate a valid step size for the validation dataset
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

model.compile(loss = 'mean_squared_error', optimizer='adam', 
              metrics=[RootMeanSquaredError(name="rmse")])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=50, verbose=1, mode='auto',
        restore_best_weights=True)

history = model.fit(train_generator, epochs=EPOCHS, steps_per_epoch=250, 
                    validation_data = val_generator, callbacks=[monitor],
                    verbose = 1, validation_steps=STEP_SIZE_VALID)

# Print elapsed time
elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Training will require a significant amount of time. Even with T4 GPU acceleration, it make require more than 1 hour to complete all 100 epochs if `EarlyStopping` doesn't kick in. 

If we wanted to,  we could zip-up the preprocessed files and store them somewhere for later use if we needed a trained neural neural network to count the number of paperclips on a piece of paper. 


## **Lesson Turn-in**

When you have completed all of the code cells, and run them in sequential order (the last code cell should be number 32) use the **File --> Print.. --> Save to PDF** to generate a PDF of your JupyterLab notebook. Save your PDF as `Class_06_2.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.